2-1. Nate 뉴스기사 제목 스크래핑하기 (필수)
https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

Image, 기사제목, 기사링크

뉴스기사의 Image를 출력 하세요 
1) Image의 절대경로와 상대 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에 



In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

def scrape_nate_news(section_url, section_name):
    """
    Nate 뉴스의 특정 섹션에서 기사 제목, 링크, 이미지를 스크래핑하는 함수. (2025-07-25 기준 수정)

    Args:
        section_url (str): 스크래핑할 뉴스 섹션의 URL.
        section_name (str): 출력할 섹션의 이름.
    """
    try:
        # 웹페이지 요청
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(section_url, headers=headers)
        response.raise_for_status()

        # UTF-8 인코딩 명시적 설정
        response.encoding = 'utf-8'

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 섹션 제목 출력 (구조 변경으로 이름 직접 출력)
        print(f"--- {section_name} ---")

        
        news_list = soup.select('div.post-list div.post-item')

        if not news_list:
            print("  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]")
            return

        for item in news_list:
            #  기사 제목 및 링크 선택자 변경: 'h5.post-tit a'
            title_element = item.select_one('h5.post-tit a')
            
            # 이미지 선택자 변경: 'div.post-thumb img'
            img_element = item.select_one('div.post-thumb img')

            if not title_element:
                continue
            
            title = title_element.text.strip()
            # href 속성이 전체 URL을 포함하므로 urljoin이 필요 없을 수 있으나, 안전을 위해 유지
            link = urljoin(section_url, title_element['href'])

            print(f"■ 제목: {title}")
            print(f"■ 링크: {link}")

            # 이미지 추출 및 출력
            if img_element and img_element.has_attr('src'):
                img_src = img_element['src']
                # 이미 전체 URL을 가지고 있으므로, '//' 처리만 확인
                if img_src.startswith('//'):
                    img_src = 'https:' + img_src
                
                try:
                    display(Image(url=img_src, width=150))
                except Exception as e:
                    print(f"  [이미지를 표시할 수 없습니다: {e}]")
            else:
                print("  [이미지 없음]")
            
            print("-" * 30)

    except requests.exceptions.RequestException as e:
        print(f"오류가 발생했습니다: {e}")

# --- 실행 부분 ---
if __name__ == '__main__':
    section_urls = {
        "최신뉴스": "https://news.nate.com/recent?mid=n0100",
        "정치": "https://news.nate.com/recent?mid=n0101",
        "경제": "https://news.nate.com/recent?mid=n0102",
        "사회": "https://news.nate.com/recent?mid=n0103",
        "세계": "https://news.nate.com/recent?mid=n0104",
        "IT/과학": "https://news.nate.com/recent?mid=n0105"
    }

    for section_name, url in section_urls.items():
        print(f"\n{'='*50}")
        print(f"'{section_name}' 뉴스 섹션 스크래핑 시작")
        print(f"{'='*50}")
        # ✨ 함수에 섹션 이름을 함께 전달
        scrape_nate_news(url, section_name)


'최신뉴스' 뉴스 섹션 스크래핑 시작
--- 최신뉴스 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]

'정치' 뉴스 섹션 스크래핑 시작
--- 정치 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]

'경제' 뉴스 섹션 스크래핑 시작
--- 경제 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]

'사회' 뉴스 섹션 스크래핑 시작
--- 사회 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]

'세계' 뉴스 섹션 스크래핑 시작
--- 세계 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]

'IT/과학' 뉴스 섹션 스크래핑 시작
--- IT/과학 ---
  [기사를 가져올 수 없습니다. 페이지 구조를 확인하세요.]


2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)
:  하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다. 
   def download_one_episode(title,no,url):

아래와 같이 호출합니다.
download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

img\일렉시드\341 디렉토리가 생성되며 , 
그 디렉토리 아래에 웹툰 image들이 다운로드 되도록 해주세요.


In [ ]:
import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 한 회차 이미지를 모두 다운로드합니다.

    :param title: 웹툰 제목 (폴더명으로 사용)
    :param no: 회차 번호 (폴더명으로 사용)
    :param url: 다운로드할 회차의 URL
    """
    # 저장할 디렉토리 경로 설정 및 생성
    # 예: img/광마회귀/160
    save_path = os.path.join('img', title, str(no))
    os.makedirs(save_path, exist_ok=True)
    print(f"'{save_path}' 디렉토리를 생성했습니다.")

    # 웹툰 페이지에 접속하기 위한 헤더 설정 (차단 방지)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': url
    }

    # 웹툰 페이지의 HTML 가져오기
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP 오류가 발생하면 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"오류: 웹 페이지를 가져올 수 없습니다. {e}")
        return

    # HTML 파싱하여 이미지 URL 추출
    soup = BeautifulSoup(response.text, 'html.parser')
    image_tags = soup.select('.wt_viewer img') # 웹툰 이미지를 담고 있는 태그 선택

    if not image_tags:
        print("오류: 이미지 태그를 찾을 수 없습니다. 페이지 구조가 변경되었을 수 있습니다.")
        return

    # 각 이미지 다운로드
    print("이미지 다운로드를 시작합니다...")
    for i, img_tag in enumerate(image_tags):
        img_url = img_tag.get('src')
        if not img_url:
            continue

        try:
            # 이미지 데이터 요청
            img_response = requests.get(img_url, headers=headers)
            img_response.raise_for_status()

            # 파일명 설정 (예: 001.jpg, 002.jpg, ...)
            file_name = f"{i+1:03d}.jpg"
            file_path = os.path.join(save_path, file_name)

            # 이미지 파일 저장
            with open(file_path, 'wb') as f:
                f.write(img_response.content)
            
            print(f"다운로드 완료: {file_name}")

        except requests.exceptions.RequestException as e:
            print(f"오류: '{img_url}' 이미지 다운로드에 실패했습니다. {e}")
            
    print(f"\n총 {len(image_tags)}개의 이미지를 '{save_path}'에 성공적으로 다운로드했습니다. 🎉")


# --- 함수 호출 예시 ---
if __name__ == "__main__":
    download_one_episode('광마회귀', 160, 'https://comic.naver.com/webtoon/detail?titleId=776601&no=160&week=fri')

'img\광마회귀\160' 디렉토리를 생성했습니다.
이미지 다운로드를 시작합니다...
다운로드 완료: 001.jpg
다운로드 완료: 002.jpg
다운로드 완료: 003.jpg
다운로드 완료: 004.jpg
다운로드 완료: 005.jpg
다운로드 완료: 006.jpg
다운로드 완료: 007.jpg
다운로드 완료: 008.jpg
다운로드 완료: 009.jpg
다운로드 완료: 010.jpg
다운로드 완료: 011.jpg
다운로드 완료: 012.jpg
다운로드 완료: 013.jpg
다운로드 완료: 014.jpg
다운로드 완료: 015.jpg
다운로드 완료: 016.jpg
다운로드 완료: 017.jpg
다운로드 완료: 018.jpg
다운로드 완료: 019.jpg
다운로드 완료: 020.jpg
다운로드 완료: 021.jpg
다운로드 완료: 022.jpg
다운로드 완료: 023.jpg
다운로드 완료: 024.jpg
다운로드 완료: 025.jpg
다운로드 완료: 026.jpg
다운로드 완료: 027.jpg
다운로드 완료: 028.jpg
다운로드 완료: 029.jpg
다운로드 완료: 030.jpg
다운로드 완료: 031.jpg
다운로드 완료: 032.jpg
다운로드 완료: 033.jpg
다운로드 완료: 034.jpg
다운로드 완료: 035.jpg
다운로드 완료: 036.jpg
다운로드 완료: 037.jpg
다운로드 완료: 038.jpg
다운로드 완료: 039.jpg
다운로드 완료: 040.jpg
다운로드 완료: 041.jpg
다운로드 완료: 042.jpg
다운로드 완료: 043.jpg
다운로드 완료: 044.jpg
다운로드 완료: 045.jpg
다운로드 완료: 046.jpg
다운로드 완료: 047.jpg
다운로드 완료: 048.jpg
다운로드 완료: 049.jpg
다운로드 완료: 050.jpg
다운로드 완료: 051.jpg
다운로드 완료: 052.jpg
다운로드 완료: 053.jpg
다운로드 완료: 054.jpg
다운로드 완료: 055.jpg
다운로드 완료: 056.jpg
